In [ ]:
import pandas as pd
CA = pd.read_csv('mapdataall.csv')

In [ ]:
CA.shape

In [ ]:
df = pd.read_fwf('national_county.txt')
df.to_csv('national_county.csv')

In [ ]:
CA.columns

In [ ]:
df = CA[['incident_acres_burned','incident_county','incident_date_extinguished', 'incident_containment', 'incident_name']]

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Exploding a Column into multiple rows

In [ ]:
type(df.incident_county[0])

In [ ]:
df.isnull().sum()

In [ ]:
df = df[pd.notnull(df['incident_county'])]

In [ ]:
df.isnull().sum()

In [ ]:
df['incident_date_extinguished'] = df['incident_date_extinguished'].fillna('Active')

In [ ]:
df.isnull().sum()

In [ ]:
new_df = pd.DataFrame(df.incident_county.str.split(', ').tolist(), index=[df.incident_acres_burned, df.incident_date_extinguished, df.incident_containment, df.incident_name]).stack()

In [ ]:
new_df.head()

In [ ]:
new_df = new_df.reset_index([0, 'incident_acres_burned', 'incident_date_extinguished','incident_containment','incident_name'])

In [ ]:
new_df.rename(columns = {0:'incident_county'}, inplace = True)

In [ ]:
new_df['incident_county'] = new_df['incident_county'] + " County"

In [ ]:
new_df['STNAME'] = 'CA'

In [ ]:
new_df.shape

In [ ]:
new_df.nlargest(10, 'incident_acres_burned')

In [ ]:
test = pd.read_csv('national_county.csv', header = None)

In [ ]:
test.rename(columns = {0: "x", 1 : "y"}, inplace = True)

In [ ]:
test2 = pd.DataFrame(test.y.str.split(',').tolist())


In [ ]:
test2.rename(columns = {0: "STNAME", 1 : "x", 2 : "y", 3 : "incident_county"}, inplace = True)

In [ ]:
test2.drop([4], axis = 1 ,inplace = True)

In [ ]:
test2["FIPS"] = test2['x'] + test2['y']

In [ ]:
test2.drop(['x', 'y'], axis = 1, inplace = True)

In [ ]:
testCA = test2[test2["STNAME"] == 'CA']

In [ ]:
testCA.head()

In [ ]:
new_df['FIPS'] = new_df.incident_county.map(testCA.set_index('incident_county')['FIPS'].to_dict())

In [ ]:
import numpy as np

In [ ]:
subset = new_df.groupby(['incident_county','FIPS']).agg({'incident_acres_burned':sum, 'incident_containment':np.mean}).reset_index()

In [ ]:
subset.shape

In [ ]:
from urllib.request import urlopen
import json
import pandas as pd
import plotly.express as px

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)


fig = px.choropleth_mapbox(subset, geojson=counties, locations='FIPS', color= 'incident_acres_burned',
                           color_continuous_scale="inferno_r",
                           hover_name = 'incident_county',
                           mapbox_style="carto-positron",
                           zoom=4.8, center = {"lat": 37.3, "lon": -119.4179},
                           opacity=0.5,
                           labels={'incident_acres_burned':'Acres Burned', 'FIPS' : 'County Code'},
                           title = 'Wildfire Damage (Acres Burned) Across CA Counties'
                          )

fig.write_html("firemap2.html")
fig.show()


In [ ]:
from urllib.request import urlopen
import json
import pandas as pd
import plotly.express as px

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)


fig = px.choropleth_mapbox(subset, geojson=counties, locations='FIPS', color= 'incident_containment',
                           color_continuous_scale="Plasma_r",
                           hover_name = 'incident_county',
                           mapbox_style="carto-positron",
                           zoom=4.8, center = {"lat": 37.3, "lon": -119.4179},
                           opacity=0.5,
                           labels={'incident_containment':'Percent Contained', 'FIPS' : 'County Code'},
                           title = 'Average Wildfire Percent Contained (By County)'
                          )

fig.write_html("firemap3.html")
fig.show()


In [ ]:
burned_by_county = subset.nlargest(10, 'incident_acres_burned')

In [ ]:
import plotly.express as px

fig = px.bar(burned_by_county, y="incident_county", x="incident_acres_burned", color="incident_county", orientation="h", 
             hover_name="incident_county", labels = {'incident_county' : 'County', 'incident_acres_burned' : 'Acres Burned'},
             color_discrete_sequence = px.colors.sequential.matter_r
            )

fig.show()

In [ ]:
duplicate = df.copy()

In [ ]:
duplicate['incident_name'] = df['incident_name'].copy().str.split('(').str[0]

In [ ]:
# duplicate['date_extinguished'] = pd.to_datetime(duplicate['incident_date_extinguished']).dt.strftime('%Y-%m-%d')

In [ ]:
duplicate.head()

In [ ]:
burned_by_name = duplicate.nlargest(10, 'incident_acres_burned')

In [ ]:
import plotly.express as px

fig = px.bar(burned_by_name, y="incident_name", x="incident_acres_burned", color="incident_name", orientation="h", 
             hover_name="incident_name", hover_data = ['incident_county','incident_date_extinguished'], labels = {'incident_date_extinguished' : 'Date Extinguished', 'incident_name' : 'Incident Name', 'incident_acres_burned' : 'Acres Burned', 'incident_county' : 'County'},
             color_discrete_sequence = px.colors.sequential.matter_r
            )

fig.show()

In [ ]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [ ]:
df.incident_name.unique()

In [ ]:
top_20_largest = pd.read_excel('top20_acres.xlsx')
top_20_largest = top_20_largest[1:21]

In [ ]:
top_20_deadliest = pd.read_excel('top20_deadliest.xlsx')
top_20_deadliest = top_20_deadliest[1:21]

In [ ]:
top_20_destructive = pd.read_excel('top20_destruction.xlsx')
top_20_destructive = top_20_destructive[1:21]

In [ ]:
top_20_largest.rename(columns = {'Unnamed: 1' : 'incident_name', 'Unnamed: 2' : 'incident_date', 
                                   'Unnamed: 3' : 'incident_county', 'Unnamed: 4' : 'incident_acres_burned', 
                                   'Unnamed: 5' : 'Structures Destroyed', 'Unnamed: 6' : 'Deaths'}, inplace = True)

In [ ]:
top_20_deadliest.rename(columns = {'Unnamed: 1' : 'incident_name', 'Unnamed: 2' : 'incident_date', 
                                   'Unnamed: 3' : 'incident_county', 'Unnamed: 4' : 'incident_acres_burned', 
                                   'Unnamed: 5' : 'Structures Destroyed', 'Unnamed: 6' : 'Deaths'}, inplace = True)

In [ ]:
top_20_destructive.rename(columns = {'Unnamed: 1' : 'incident_name', 'Unnamed: 2' : 'incident_date', 
                                   'Unnamed: 3' : 'incident_county', 'Unnamed: 4' : 'incident_acres_burned', 
                                   'Unnamed: 5' : 'Structures Destroyed', 'Unnamed: 6' : 'Deaths'}, inplace = True)

In [ ]:
top_20_largest['incident_cause'] = top_20_largest['incident_name'].str.split('(').str.get(1)
top_20_largest['incident_cause'] = top_20_largest['incident_cause'].str.extract('([a-zA-Z ]+)', expand=False).str.strip()
top_20_largest['incident_name'] = top_20_largest['incident_name'].str.extract('([a-zA-Z ]+)', expand=False).str.strip()
top_20_largest['incident_county'] = top_20_largest['incident_county'].str.replace('\n', '')

In [ ]:
top_20_deadliest['incident_cause'] = top_20_deadliest['incident_name'].str.split('(').str.get(1)
top_20_deadliest['incident_cause'] = top_20_deadliest['incident_cause'].str.extract('([a-zA-Z ]+)', expand=False).str.strip()
top_20_deadliest['incident_name'] = top_20_deadliest['incident_name'].str.extract('([a-zA-Z ]+)', expand=False).str.strip()

In [ ]:
top_20_destructive['incident_cause'] = top_20_destructive['incident_name'].str.split('(').str.get(1)
top_20_destructive['incident_cause'] = top_20_destructive['incident_cause'].str.extract('([a-zA-Z ]+)', expand=False).str.strip()
top_20_destructive['incident_name'] = top_20_destructive['incident_name'].str.extract('([a-zA-Z ]+)', expand=False).str.strip()

In [ ]:
top_20_largest['incident_cause'] = top_20_largest['incident_cause'].replace(['Undetermined'],'Unknown')

In [ ]:
from collections import Counter

labels = top_20_largest['incident_cause']
largest_labels = list(top_20_largest['incident_cause'].unique())
largest_values = list(Counter(labels).values())

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

colors = ['gold', 'mediumturquoise', 'OrangeRed', 'LightSkyBlue', 'MediumOrchid']
# pull is given as a fraction of the pie radius

fig = make_subplots(rows=1, cols=1, specs=[[{'type':'domain'}]])

fig.add_trace(go.Pie(labels=largest_labels, values=largest_values, pull=[0, 0, 0.2, 0, 0], name = 'Wildfire Cause', sort = False),
              1, 1)
fig.update_traces(hoverinfo="label+percent+name", textfont_size = 16, 
                 marker=dict(colors=colors, line=dict(color='#000000', width=2)))

fig.update_layout(title = 'Causes of the Top 20 Largest California Wildfires')
fig.show()

In [ ]:
top_20_deadliest['incident_cause'] = top_20_deadliest['incident_cause'].replace(['Undetermined'],'Unknown')
top_20_deadliest['incident_cause'] = top_20_deadliest['incident_cause'].replace(['Arson', 'Vehicle'],'Human Related')
top_20_deadliest['incident_cause'] = top_20_deadliest['incident_cause'].replace(['Powerline', 'Power Lines'],'Powerlines')

In [ ]:
labels = top_20_deadliest['incident_cause']
deadliest_labels = list(top_20_deadliest['incident_cause'].unique())
deadliest_values = list(Counter(labels).values())

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

colors = ['gold', 'mediumturquoise', 'darkorange', 'LightSkyBlue', 'MediumOrchid', 'OrangeRed', 'Pink']
# pull is given as a fraction of the pie radius

fig = make_subplots(rows=1, cols=1, specs=[[{'type':'domain'}]])

fig.add_trace(go.Pie(labels=deadliest_labels, values=deadliest_values, pull=[0, 0, 0, 0, 0, 0.2, 0], name = 'Wildfire Cause', sort = False),
              1, 1)
fig.update_traces(hoverinfo="label+percent+name", textfont_size = 16, 
                 marker=dict(colors=colors, line=dict(color='#000000', width=2)))

fig.update_layout(title = 'Causes of the Top 20 Deadliest California Wildfires')

fig.show()

In [ ]:
top_20_destructive['incident_cause'] = top_20_destructive['incident_cause'].replace(['Undetermined'],'Unknown')
top_20_destructive['incident_cause'] = top_20_destructive['incident_cause'].replace(['Arson', 'Vehicle'],'Human Related')
top_20_destructive['incident_cause'] = top_20_destructive['incident_cause'].replace(['Powerline', 'Power Lines'],'Powerlines')

In [ ]:
labels = top_20_destructive['incident_cause']
destructive_labels = list(top_20_destructive['incident_cause'].unique())
destructive_values = list(Counter(labels).values())

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

colors = ['gold', 'mediumturquoise', 'darkorange', 'OrangeRed', 'MediumOrchid', 'LightSkyBlue', 'Pink']
# pull is given as a fraction of the pie radius

fig = make_subplots(rows=1, cols=1, specs=[[{'type':'domain'}]])

fig.add_trace(go.Pie(labels=destructive_labels, values=destructive_values, pull=[0, 0, 0, 0.2, 0, 0, 0], name = 'Wildfire Cause', sort = False),
              1, 1)
fig.update_traces(hoverinfo="label+percent+name", textfont_size = 16, 
                 marker=dict(colors=colors, line=dict(color='#000000', width=2)))

fig.update_layout(title = 'Causes of the Top 20 Most Destructive California Wildfires')

fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots



fig = make_subplots(rows=1, cols=3, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(labels=largest_labels, values=largest_values, pull=[0, 0, 0.2, 0, 0],
                     name = 'Cause', sort = False), 1, 1)
fig.add_trace(go.Pie(labels=deadliest_labels, values=deadliest_values, pull=[0, 0, 0, 0, 0, 0.2, 0], 
                     name='Cause', sort = False), 1, 2)
fig.add_trace(go.Pie(labels=destructive_labels, values=destructive_values, pull=[0, 0, 0, 0.2, 0, 0, 0], 
                     name='Cause', sort = False,
                     marker = {'colors' : ['gold', 'DarkTurquoise', 'BurlyWood', 
                                          'OrangeRed', 'MediumOrchid', 'LightSkyBlue',
                                          'Pink']}), 1, 3)


fig.update_traces(hoverinfo="label+percent+name", textfont_size = 12,
                  marker=dict(line=dict(color='#000000', width=2)))

fig.update_layout(
    title = 'Causes of the Largest, Deadliest, and Most Destructive California Wildfires',
    annotations=[dict(text='Causes: Top 20 Largest Fires', x=-0.004, y=0.85, font_size=13, showarrow=False),
                 dict(text='Causes: Top 20 Deadliest Fires', x=0.51, y=0.85, font_size=13, showarrow=False),
                 dict(text='Causes: Top 20 Most Destructive', x=1.02, y=0.85, font_size=13, showarrow=False)])

fig.show()

In [ ]:
i1 = np.intersect1d(top_20_largest.incident_name, np.intersect1d(top_20_deadliest.incident_name, top_20_destructive.incident_name))

In [ ]:
i1

In [ ]:
i2 = np.intersect1d(top_20_largest.incident_name, top_20_deadliest.incident_name)

In [ ]:
i2

In [ ]:
i3 = np.intersect1d(top_20_largest.incident_name, top_20_destructive.incident_name)

In [ ]:
i3

In [ ]:
i4 = np.intersect1d(top_20_deadliest.incident_name, top_20_destructive.incident_name)

In [ ]:
i4

In [ ]:
i5 = list(i1) + list(i2) + list(i3) + list(i4)

In [ ]:
i6 = list(set(i5))

In [ ]:
i6

In [ ]:
data = pd.DataFrame(columns = ['TOP 20 LARGEST', 'TOP 20 DEADLIEST', 'TOP 20 MOST DESTRUCTIVE'], index = i6)

In [ ]:
data

In [ ]:
data.loc['OLD'] = [0,1,1]
data.loc['TUNNEL'] = [0,1,1]
data.loc['NORTH COMPLEX'] = [1,1,1]
data.loc['LNU LIGHTNING COMPLEX'] = [1,0,1]
data.loc['CARR'] = [1,1,1]
data.loc['AUGUST COMPLEX'] = [1,0,1]
data.loc['THOMAS'] = [1,0,1]
data.loc['ATLAS'] = [0,1,1]
data.loc['CREEK FIRE'] = [1,0,1]
data.loc['CEDAR'] = [1,1,1]
data.loc['TUBBS'] = [0,1,1]
data.loc['CAMP FIRE'] = [0,1,1]
data.loc['WITCH'] = [1,0,1]

In [ ]:
new_index = []
for i in data.index:
    if 'COMPLEX' in i or 'FIRE' in i:
        new_index.append(i)
    else:
        new_index.append(str(i) + ' FIRE')
        

In [ ]:
data.index = new_index

In [ ]:
data

In [ ]:
from networkx.algorithms import bipartite

In [ ]:
from networkx.algorithms import nx

B = nx.Graph()
B.add_nodes_from(data.index, bipartite=0)
B.add_nodes_from(data.columns, bipartite=1)

s = data.stack()
B.add_edges_from(s[s==1].index)


In [ ]:
color = bipartite.color(B)
color_dict = {0:'g',1:'r'}
color_list = [color_dict[i[1]] for i in B.nodes.data('bipartite')]

In [ ]:
from matplotlib.pyplot import figure, text

figure(figsize=(12,12))

top = nx.bipartite.sets(B)[0]
pos = nx.bipartite_layout(B, top)


nx.draw(B, pos=pos, with_labels=False, 
        node_size=1000, node_color = color_list)

# count = 0
# for node, (x, y) in pos.items():
#     if node == 'TOP 20 MOST DESTRUCTIVE':
#         text(x,y-0.038,s=node, bbox=dict(facecolor=color_list[count], alpha=0.3), fontsize = 12,
#              horizontalalignment='center')
#     elif 'TOP 20' in node:
#         text(x+0.05,y+0.03,s=node, bbox=dict(facecolor=color_list[count], alpha=0.3), fontsize = 12,
#              horizontalalignment='center')
#     else:
#         text(x,y+0.03,s=node, bbox=dict(facecolor=color_list[count],alpha=0.3),fontsize = 12,horizontalalignment='center')
#     count += 1
 
count = 0
for node, (x, y) in pos.items():
    if node == 'TOP 20 MOST DESTRUCTIVE':
        text(x+0.01,y-0.062,s=node, bbox=dict(facecolor=color_list[count], alpha=0.3), fontsize = 12,
             horizontalalignment='center', rotation = 13)
    elif node == 'TOP 20 LARGEST':
        text(x+0.01,y-0.055,s=node, bbox=dict(facecolor=color_list[count], alpha=0.3), fontsize = 12,
             horizontalalignment='center', rotation = 15)
    elif node == 'TOP 20 DEADLIEST':
        text(x+0.02,y-0.058,s=node, bbox=dict(facecolor=color_list[count], alpha=0.3), fontsize = 12,
             horizontalalignment='center', rotation = 17)
    else:
        text(x,y+0.03,s=node, bbox=dict(facecolor=color_list[count],alpha=0.3),fontsize = 12,horizontalalignment='center')
    count += 1

In [ ]:
top_20_largest.head()

In [ ]:
top_20_deadliest.head()

In [ ]:
top_20_destructive.head()